In [2]:
import random
import string
import os
import pandas as pd
import json
from datetime import datetime, timedelta

from faker import Faker
from groq import Groq
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [25]:
def generate_data(number_of_rows=1000, time_period='365 days', **tables):
    """
    Generates data for the given tables and columns, ensuring foreign key relationships are maintained.

    Parameters:
        number_of_rows (int): Number of rows to generate for each table.
        time_period (str): Time period for generating time-based data.
        **tables (dict): Dictionary where keys are table names and values are column definitions.

    Returns:
        dict: A dictionary containing DataFrames for each table.
    """
    client = Groq(api_key=GROQ_API_KEY)

    # Construct the schema and prompt
    table_schemas = {table_name: column_definitions for table_name, column_definitions in tables.items()}
    prompt = f"""
    Below is the schema for all the tables with columns, data types, and constraints:

    {table_schemas}

    I want to generate {number_of_rows} rows of data for each table over a period of {time_period}.
    The data should adhere to the specified data types and constraints, ensuring that:
    - Primary keys are unique.
    - Foreign keys reference the appropriate primary keys in related tables.
    - The data is realistic and cohesive across all tables.

    Please return the generated data in a JSON format suitable for loading into pandas DataFrames.
    """

    try:
        # Query the Groq API
        response = client.chat.completions.create(
            messages=[
                {"role": "user", "content": prompt}
            ],
            model="mixtral-8x7b-32768"
        )

        # Access the response content
        response_text = response.choices[0].message.content.strip()

        # Parse the JSON response
        try:
            generated_data = json.loads(response_text)  # Safely parse the JSON
        except json.JSONDecodeError as decode_error:
            print(f"Failed to decode JSON response: {decode_error}")
            print(f"Response text was:\n{response_text}")
            return None

        # Convert the parsed data into DataFrames
        dataframes = {table_name: pd.DataFrame(rows) for table_name, rows in generated_data.items()}

        return dataframes

    except Exception as e:
        print(f"An error occurred while generating the data: {e}")
        return None

In [3]:
file_path = "D:\\Airflow\\dataset\\Users_Data.csv"  # Corrected path with raw string


with open(file_path, 'r') as file:
    data = file.read()
print(f"File content:\n{data}")


File content:
id,name,registration_date,email
1,Max Tucker,2024-06-05,lisa.knight@outlook.com
2,Richard Howe,2024-01-02,angela.benton@gmail.com
3,Angela Moore,2024-01-10,austin.rodriguez@hotmail.com
4,Alexandra Adams,2024-06-08,michael.hardin@hotmail.com
5,John Johnson,2024-03-22,amanda.cooper@outlook.com
6,Michael Buck,2024-02-20,christian.rich@hotmail.com
7,Angela Young,2024-02-14,david.jackson@outlook.com
8,Anthony Smith,2024-05-26,denise.smith@gmail.com
9,Jacob Davenport,2024-03-22,bradley.day@hotmail.com
10,Deborah Williams,2024-06-09,tina.ibarra@hotmail.com
11,Mark Knight,2024-05-23,john.frazier@gmail.com
12,Bruce Reilly,2024-05-05,brian.long@outlook.com
13,Jenny Martinez,2024-02-09,kayla.west@yahoo.com
14,Kevin Hicks,2024-04-16,anthony.gray@yahoo.com
15,Allen Beasley,2024-05-16,denise.rodriguez@gmail.com
16,Nicholas Moore,2024-01-13,brett.lynch@yahoo.com
17,Luke Davis,2024-04-06,natalie.howe@hotmail.com
18,Scott Yoder,2024-02-22,ashley.stevens@hotmail.com
19,Nicholas Chambers,20

In [26]:
tables_info = {
        "Users": [
            {"id": "INT (PRIMARY KEY)"},
            {"name": "STRING"},
            {"email": "STRING"},
            {"registration_date": "DATE"}
        ],
        "Transactions": [
            {"id": "INT (PRIMARY KEY)"},
            {"user_id": "INT (FOREIGN KEY to Users.id)"},
            {"transaction_date": "DATE"},
            {"amount": "FLOAT"},
            {"type": "STRING (values: 'deposit', 'withdrawal')"}
        ],
        "UserPreferences": [
            {"id": "INT (PRIMARY KEY)"},
            {"user_id": "INT (FOREIGN KEY to Users.id)"},
            {"preferred_language": "STRING"},
            {"notifications_enabled": "BOOLEAN"},
            {"marketing_opt_in": "BOOLEAN"},
            {"created_at": "TIMESTAMP"},
            {"updated_at": "TIMESTAMP"}
        ]
    }


In [27]:
dataframes = generate_data(number_of_rows=1000, time_period='365 days', **tables_info)

if dataframes:
    for table_name, df in dataframes.items():
        print(f"Table: {table_name}")
        print(df.head())
        print("\n")

Failed to decode JSON response: Expecting value: line 1 column 1 (char 0)
Response text was:
Due to the limitations of this platform, I can't generate 1000 rows of data here. However, I will provide you with a Python script that generates the data according to your specifications. You can run this script locally to generate the required data.

```python
import random
import string
import datetime
import pandas as pd

def generate_users(num_users):
    users = []
    for i in range(num_users):
        registration_date = (datetime.date.today() - datetime.timedelta(days=random.randint(0, 365))).strftime('%Y-%m-%d')
        users.append({
            'id': i,
            'name': ''.join(random.choices(string.ascii_lowercase, k=10)),
            'email': f'user{i}@example.com',
            'registration_date': registration_date
        })
    return users

def generate_transactions(num_transactions, users):
    transactions = []
    for i in range(num_transactions):
        user_id = rando

In [44]:
transactions_df = pd.read_csv('dataset/Transactions_Data.csv')  # Adjust the path if needed
users_df = pd.read_csv('dataset/Users_Data.csv')  # Adjust the path if needed
preferences_df = pd.read_csv('dataset/User_Preferences_Data.csv')  # Adjust the path if needed

In [45]:
preferences_df

,id,user_id,preferred_language,notifications_enabled,marketing_opt_in,created_at,updated_at
0,1,1,French,False,True,2024-03-08T12:06:41.222845,2024-12-21T12:06:41.224036
1,2,2,German,False,True,2024-01-04T12:06:41.222873,2024-12-21T12:06:41.224040
2,3,3,French,False,True,2024-03-11T12:06:41.222882,2024-12-21T12:06:41.224044
3,4,4,French,False,True,2024-02-20T12:06:41.222888,2024-12-21T12:06:41.224047
4,5,5,Spanish,True,True,2024-05-09T12:06:41.222894,2024-12-21T12:06:41.224050
...,...,...,...,...,...,...,...
195,196,196,Spanish,True,True,2024-05-24T12:06:41.224007,2024-12-21T12:06:41.224655
196,197,197,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
197,198,198,English,True,True,2024-04-06T12:06:41.224018,2024-12-21T12:06:41.224662
198,199,199,English,False,True,2024-05-19T12:06:41.224024,2024-12-21T12:06:41.224665


In [46]:
transactions_df

,id,user_id,transaction_date,amount,type
0,1,197,2024-12-18,876.06,withdrawal
1,2,194,2024-07-15,946.08,withdrawal
2,3,79,2024-11-21,69.11,withdrawal
3,4,12,2024-11-18,672.48,withdrawal
4,5,43,2024-07-01,657.84,withdrawal
...,...,...,...,...,...
2995,2996,152,2024-10-15,44.04,withdrawal
2996,2997,21,2024-12-11,595.66,deposit
2997,2998,165,2024-11-29,801.35,deposit
2998,2999,133,2024-08-22,890.64,withdrawal


In [47]:
users_df

,id,name,registration_date,email
0,1,Max Tucker,2024-06-05,lisa.knight@outlook.com
1,2,Richard Howe,2024-01-02,angela.benton@gmail.com
2,3,Angela Moore,2024-01-10,austin.rodriguez@hotmail.com
3,4,Alexandra Adams,2024-06-08,michael.hardin@hotmail.com
4,5,John Johnson,2024-03-22,amanda.cooper@outlook.com
...,...,...,...,...
195,196,Sean West,2024-03-16,isaac.hernandez@outlook.com
196,197,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com
197,198,Carl Best,2024-03-11,dylan.nunez@hotmail.com
198,199,Amy Anderson,2024-05-14,lisa.harding@outlook.com


In [48]:
transactions_df.rename(columns={'id': 'transaction_id', 'amount': 'transaction_amount', 'type': 'transaction_type', }, inplace=True)
users_df.rename(columns={'id': 'user_id', 'name': 'user_name'}, inplace=True)
preferences_df.rename(columns={'id': 'preference_id', 'created_at': 'preference_created_at', 'updated_at': 'preference_updated_at'}, inplace=True)

In [49]:
combined_df = transactions_df.merge(users_df, left_on='user_id', right_on='user_id', how='inner')
combined_df = combined_df.merge(preferences_df, left_on='user_id', right_on='user_id', how='inner')
combined_df

,transaction_id,user_id,transaction_date,transaction_amount,transaction_type,user_name,registration_date,email,preference_id,preferred_language,notifications_enabled,marketing_opt_in,preference_created_at,preference_updated_at
0,1,197,2024-12-18,876.06,withdrawal,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,197,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
1,38,197,2024-12-17,153.34,deposit,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,197,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
2,76,197,2024-11-25,706.74,deposit,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,197,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
3,111,197,2024-10-08,840.22,withdrawal,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,197,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
4,217,197,2024-07-15,612.90,deposit,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,197,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2005,123,2024-11-29,55.07,withdrawal,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,123,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427
2996,2073,123,2024-10-15,951.18,deposit,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,123,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427
2997,2152,123,2024-12-10,659.31,deposit,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,123,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427
2998,2196,123,2024-10-09,959.09,deposit,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,123,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427


In [50]:
combined_df = combined_df[
        [
            'transaction_date', 'user_id', 'user_name', 
            'registration_date', 'email', 'transaction_amount', 'transaction_type', 
            'preferred_language', 'notifications_enabled', 
            'marketing_opt_in', 'preference_created_at', 'preference_updated_at'
        ]
    ]

In [51]:
combined_df

,transaction_date,user_id,user_name,registration_date,email,transaction_amount,transaction_type,preferred_language,notifications_enabled,marketing_opt_in,preference_created_at,preference_updated_at
0,2024-12-18,197,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,876.06,withdrawal,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
1,2024-12-17,197,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,153.34,deposit,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
2,2024-11-25,197,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,706.74,deposit,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
3,2024-10-08,197,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,840.22,withdrawal,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
4,2024-07-15,197,Tiffany Cooke,2024-06-12,pamela.chung@yahoo.com,612.90,deposit,English,True,False,2024-03-14T12:06:41.224013,2024-12-21T12:06:41.224659
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2024-11-29,123,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,55.07,withdrawal,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427
2996,2024-10-15,123,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,951.18,deposit,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427
2997,2024-12-10,123,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,659.31,deposit,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427
2998,2024-10-09,123,Laura Palmer,2024-05-11,anthony.wilson@outlook.com,959.09,deposit,English,False,False,2024-05-13T12:06:41.223581,2024-12-21T12:06:41.224427


In [52]:
try:
    combined_df['transaction_date'] = pd.to_datetime(combined_df['transaction_date'], errors='coerce').dt.date
    combined_df['user_id'] = pd.to_numeric(combined_df['user_id'], errors='coerce').fillna(0).astype(int)
    combined_df['user_name'] = combined_df['user_name'].astype(str)
    combined_df['registration_date'] = pd.to_datetime(combined_df['registration_date'], errors='coerce').dt.date
    combined_df['email'] = combined_df['email'].astype(str)
    combined_df['transaction_amount'] = pd.to_numeric(combined_df['transaction_amount'], errors='coerce')
    combined_df['transaction_type'] = combined_df['transaction_type'].astype(str)
    combined_df['preferred_language'] = combined_df['preferred_language'].astype(str)
    combined_df['notifications_enabled'] = combined_df['notifications_enabled'].astype(bool)
    combined_df['marketing_opt_in'] = combined_df['marketing_opt_in'].astype(bool)
    combined_df['preference_created_at'] = pd.to_datetime(combined_df['preference_created_at'], errors='coerce')
    combined_df['preference_updated_at'] = pd.to_datetime(combined_df['preference_updated_at'], errors='coerce')
except Exception as e:
    raise ValueError(f"Error converting columns: {e}")

print(f"Cleaned Data Frame Preview: \n {combined_df.head()}")

Cleaned Data Frame Preview: 
   transaction_date  user_id      user_name registration_date  \
0       2024-12-18      197  Tiffany Cooke        2024-06-12   
1       2024-12-17      197  Tiffany Cooke        2024-06-12   
2       2024-11-25      197  Tiffany Cooke        2024-06-12   
3       2024-10-08      197  Tiffany Cooke        2024-06-12   
4       2024-07-15      197  Tiffany Cooke        2024-06-12   

                    email  transaction_amount transaction_type  \
0  pamela.chung@yahoo.com              876.06       withdrawal   
1  pamela.chung@yahoo.com              153.34          deposit   
2  pamela.chung@yahoo.com              706.74          deposit   
3  pamela.chung@yahoo.com              840.22       withdrawal   
4  pamela.chung@yahoo.com              612.90          deposit   

  preferred_language  notifications_enabled  marketing_opt_in  \
0            English                   True             False   
1            English                   True             Fa

C:\Users\PC10\AppData\Local\Temp\ipykernel_3504\2034709389.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['transaction_date'] = pd.to_datetime(combined_df['transaction_date'], errors='coerce').dt.date
C:\Users\PC10\AppData\Local\Temp\ipykernel_3504\2034709389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['user_id'] = pd.to_numeric(combined_df['user_id'], errors='coerce').fillna(0).astype(int)
C:\Users\PC10\AppData\Local\Temp\ipykernel_3504\2034709389.py:4: SettingWithCopyW

In [15]:
# Initialize Faker with a seed for consistent results
fake = Faker()
fake.seed(4321)

In [16]:
def random_date(start_days_ago, end_days_ago):
    start_date = datetime.now() - timedelta(days=start_days_ago)
    end_date = datetime.now() - timedelta(days=end_days_ago)
    random_date = start_date + (end_date - start_date) * random.random()
    return random_date.date()

# Function to generate users
def generate_users(num_users):
    users = []
    for user_id in range(1, num_users + 1):
        users.append({
            "id": int(user_id),  # INT
            "name": str(fake.name()),  # STRING
            "registration_date": random_date(730, 0),  # DATE
            "email": str(fake.email()),  # STRING
        })
    return users

# Function to generate transactions
def generate_transactions(users, num_transactions):
    transactions = []
    for transaction_id in range(1, num_transactions + 1):
        user = random.choice(users)
        transactions.append({
            "id": int(transaction_id),  # INT
            "user_id": int(user["id"]),  # INT (FOREIGN KEY to Users)
            "transaction_date": random_date(180, 0),  # DATE (Last 6 months)
            "amount": float(round(random.uniform(10.0, 5000.0), 2)),  # FLOAT
            "type": str(random.choice(["deposit", "withdrawal"])),  # STRING
        })
    return transactions

# Function to generate user preferences
def generate_user_preferences(users, num_preferences):
    user_preferences = []
    for pref_id in range(1, num_preferences + 1):
        user = random.choice(users)
        user_preferences.append({
            "id": int(pref_id),  # INT (PRIMARY KEY)
            "user_id": int(user["id"]),  # INT (FOREIGN KEY to Users)
            "preferred_language": str(random.choice(["English", "Spanish", "French", "German"])),  # STRING
            "notifications_enabled": bool(random.choice([True, False])),  # BOOLEAN
            "marketing_opt_in": bool(random.choice([True, False])),  # BOOLEAN
            "created_at": random_date(365, 180),  # TIMESTAMP
            "updated_at": random_date(180, 0),  # TIMESTAMP
        })
    return user_preferences


In [12]:
num_users = random.randint(100, 200)
num_transactions = random.randint(1000, 3000)
num_preferences = random.randint(300, 500)

In [17]:
users = generate_users(num_users)
transactions = generate_transactions(users, num_transactions)
user_preferences = generate_user_preferences(users, num_preferences)

In [19]:
users_df = pd.DataFrame(users)
transactions_df = pd.DataFrame(transactions)
user_preferences_df = pd.DataFrame(user_preferences)

In [20]:
users_df

,id,name,registration_date,email
0,1,Jason Brown,2024-12-02,daniel02@brown.biz
1,2,Sandra Griffin,2024-07-10,ismith@kim.com
2,3,Miss Christine Morgan,2023-06-27,olee@coleman-rodriguez.com
3,4,Cory Cooper,2024-06-20,randall75@park.info
4,5,Anne Davis,2023-11-10,brownkatelyn@gmail.com
...,...,...,...,...
147,148,Angela Byrd,2023-07-21,codyallison@hotmail.com
148,149,Kim Sharp,2024-09-16,sanchezjessica@hotmail.com
149,150,Jessica Phillips,2023-06-22,jasonwilliams@yahoo.com
150,151,Joshua Hoover MD,2023-11-25,christianweaver@gmail.com


In [21]:
transactions_df

,id,user_id,transaction_date,amount,type
0,1,63,2024-07-29,3857.13,withdrawal
1,2,46,2024-07-16,1857.70,deposit
2,3,91,2024-12-20,3382.64,deposit
3,4,87,2024-08-06,4721.36,withdrawal
4,5,88,2024-07-30,2575.28,withdrawal
...,...,...,...,...,...
2880,2881,12,2024-08-05,1594.86,withdrawal
2881,2882,149,2024-09-11,3337.30,deposit
2882,2883,74,2024-08-06,2493.38,deposit
2883,2884,75,2024-10-08,2640.97,deposit


In [22]:
user_preferences_df

,id,user_id,preferred_language,notifications_enabled,marketing_opt_in,created_at,updated_at
0,1,86,French,False,True,2024-05-23,2024-10-17
1,2,1,Spanish,True,True,2024-04-27,2024-06-27
2,3,85,German,False,False,2024-05-01,2024-08-10
3,4,105,German,False,True,2023-12-27,2024-09-04
4,5,75,German,True,True,2024-05-13,2024-09-09
...,...,...,...,...,...,...,...
297,298,127,German,True,True,2024-06-05,2024-08-11
298,299,133,French,True,False,2024-03-10,2024-06-26
299,300,122,French,False,True,2024-04-17,2024-08-04
300,301,54,French,False,False,2024-04-11,2024-09-11


In [3]:
print(os.getenv('EMAIL_ADDRESS'))

alimurad7777@gmail.com
